In [ ]:
import isaacgym
import isaacgymenvs
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator

from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder

from omegaconf import DictConfig, OmegaConf

import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cfg = OmegaConf.load("cfg/config.yaml")
cfg.task_name="TrifingerNYU"
cfg.num_envs=1
cfg.task=OmegaConf.load("cfg/task/TrifingerNYU.yaml")

In [ ]:
device = cfg.sim_device

In [ ]:
def create_env_thunk(**kwargs):
        envs = isaacgymenvs.make(
            
            cfg.seed,
            cfg.task,
            cfg.num_envs,
            cfg.sim_device,
            cfg.rl_device,
            headless=True,
            cfg=cfg,
            **kwargs,
        )
        return envs

In [ ]:
envs = create_env_thunk()

In [ ]:
# get fingertip states
N = 500
action_buffer = torch.zeros(N, 9).to(device)
ftip_pos_buffer = torch.zeros(N, 9).to(device)
ftip_vel_buffer = torch.zeros(N, 9).to(device)
jacobian_buffer = torch.zeros(N, 9, 9).to(device)
dof_vel_buffer = torch.zeros(N, 9).to(device)

for n in range(N):
    action = torch.rand(1, 9).to(device) * 2 - 1
    obs, rwds, resets, info = envs.step(torch.rand(1, 9).to(cfg.sim_device) * 2 - 1)
    action_buffer[n] = action
    
    q = envs._dof_position
    dq = envs._dof_velocity
    dof_vel_buffer[n] = dq[0]
    
    fingertip_state = envs._rigid_body_state[:, envs._fingertip_indices]
    fingertip_position = fingertip_state[:, :, 0:3].reshape(envs.num_envs, 9)
    fingertip_velocity = fingertip_state[:, :, 7:10].reshape(envs.num_envs, 9)
    ftip_pos_buffer[n] = fingertip_position[0]
    ftip_vel_buffer[n] = fingertip_velocity[0]
    
    fid = [5, 12, 19]
    jacobian_fingertip_linear = envs._jacobian[:, fid, :3, :]
    jacobian_fingertip_linear = jacobian_fingertip_linear.view(
                    envs.num_envs, 
                    3 * envs._dims.NumFingers.value, 
                    envs._dims.GeneralizedCoordinatesDim.value)
    jacobian_buffer[n] = jacobian_fingertip_linear[0]

In [ ]:
ftip_pos_buffer_np = ftip_pos_buffer.cpu().numpy()
ftip_vel_buffer_np = ftip_vel_buffer.cpu().numpy()
jdq = torch.einsum('bij, bj -> bi', jacobian_buffer, dof_vel_buffer)
jdq_np = jdq.cpu().numpy()

In [ ]:
dt = 0.02
ftip_vel_numdiff = np.diff(ftip_pos_buffer_np, axis=0) / dt
ftip_vel_numdiff = np.vstack((np.zeros((1, 9)), ftip_vel_numdiff))

In [ ]:
d = 0
# plt.plot(ftip_vel_numdiff[:, d])
plt.plot(ftip_vel_buffer_np[:, d])
plt.plot(jdq_np[:, d], color='orange', ls='-.')

In [ ]:
import trifinger_simulation
import numpy as np
import pinocchio as pin

In [ ]:
platform = trifinger_simulation.TriFingerPlatform()
trifinger = platform.simfinger
rmodel = trifinger.kinematics.robot_model
rdata = trifinger.kinematics.data
ftip_id = trifinger.kinematics.tip_link_ids

In [ ]:
q1_np = q1[0].cpu().numpy()
dq1_np = dq1[0].cpu().numpy()

In [ ]:
trifinger.kinematics.forward_kinematics(q_np)

In [ ]:
pin.computeJointJacobians(rmodel, rdata, q_np)
pin.framesForwardKinematics(rmodel, rdata, q_np)
J_ = []
for i in range(3):
    J_.append(pin.getFrameJacobian(
        rmodel,
        rdata,
        ftip_id[i],
        pin.ReferenceFrame.LOCAL_WORLD_ALIGNED,
    )[:3, :])
J = np.vstack(J_)

In [ ]:
J @ dq_np # pinocchio, local world aligned

In [ ]:
jacobian_fingertip_linear[0] @ dq[0] # isaac

In [ ]:
fingertip_velocity # isaac direct readings